In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pcdet.config import cfg, cfg_from_list, cfg_from_yaml_file, log_config_to_file
from pcdet.models.backbones_3d.pointnet2_backbone import PointNet2MSG_FPS
from pcdet.models.backbones_3d.pfe import VoteModule
from pcdet.models import build_network
from pcdet.models.dense_heads.point_head_box_3dssd import PointHeadBox3DSSD
from pcdet.datasets import build_dataloader
from pcdet.utils import common_utils
import torch
import numpy as np

In [3]:
cfg_file = "/home/yzy/PycharmProjects/OpenPCDet/tools/cfgs/kitti_models/3dssd_imvote.yaml"
cfg_from_yaml_file(cfg_file, cfg)
print(cfg)
dataset, dataloader, sampler = build_dataloader(cfg['DATA_CONFIG'],
                                                class_names=['Car', 'Pedestrian', 'Cyclist'],
                                                batch_size=2,
                                                dist=False)

{'ROOT_DIR': PosixPath('/home/yzy/PycharmProjects/OpenPCDet'), 'LOCAL_RANK': 0, 'CLASS_NAMES': ['Car', 'Pedestrian', 'Cyclist'], 'DATA_CONFIG': {'DATASET': 'KittiDatasetImVote', 'DATA_PATH': '/home/yzy/PycharmProjects/OpenPCDet/data/kitti', 'POINT_CLOUD_RANGE': [0, -40, -3, 70.4, 40, 1], 'DATA_SPLIT': {'train': 'train', 'test': 'val'}, 'INFO_PATH': {'train': ['kitti_infos_train.pkl'], 'test': ['kitti_infos_val.pkl']}, 'FOV_POINTS_ONLY': True, 'DATA_AUGMENTOR': {'DISABLE_AUG_LIST': ['placeholder'], 'AUG_CONFIG_LIST': [{'NAME': 'gt_sampling', 'USE_ROAD_PLANE': True, 'DB_INFO_PATH': ['kitti_dbinfos_train.pkl'], 'PREPARE': {'filter_by_min_points': ['Car:5', 'Pedestrian:5', 'Cyclist:5'], 'filter_by_difficulty': [-1]}, 'SAMPLE_GROUPS': ['Car:20', 'Pedestrian:15', 'Cyclist:15'], 'NUM_POINT_FEATURES': 4, 'DATABASE_WITH_FAKELIDAR': False, 'REMOVE_EXTRA_WIDTH': [0.0, 0.0, 0.0], 'LIMIT_WHOLE_SCENE': True}, {'NAME': 'random_world_flip', 'ALONG_AXIS_LIST': ['x']}, {'NAME': 'random_world_rotation', 

In [4]:
dataset

In [5]:
model = build_network(model_cfg=cfg['MODEL'], num_class=3, dataset=dataset)
model.cuda()

{'Detector3DTemplate': <class 'pcdet.models.detectors.detector3d_template.Detector3DTemplate'>, 'SECONDNet': <class 'pcdet.models.detectors.second_net.SECONDNet'>, 'PartA2Net': <class 'pcdet.models.detectors.PartA2_net.PartA2Net'>, 'PVRCNN': <class 'pcdet.models.detectors.pv_rcnn.PVRCNN'>, 'PointPillar': <class 'pcdet.models.detectors.pointpillar.PointPillar'>, 'PointRCNN': <class 'pcdet.models.detectors.point_rcnn.PointRCNN'>, 'SSD3D': <class 'pcdet.models.detectors.ssd3d.SSD3D'>, 'SSD3D_imvote': <class 'pcdet.models.detectors.ssd3d_imvote.SSD3D_imvote'>}
[4, 16, 16, 32]
[4, 16, 16, 32]
[4, 32, 32, 64]
[67, 64, 64, 128]
[67, 64, 64, 128]
[67, 64, 96, 128]
[131, 128, 128, 256]
[131, 128, 192, 256]
[131, 128, 256, 256]
[[256, 256, 256, 512], [256, 256, 512, 1024]]
[259, 256, 256, 512]
[259, 256, 512, 1024]
[[256, 256, 256, 512], [256, 256, 512, 1024]]
[259, 256, 256, 512]
[259, 256, 512, 1024]
[[256, 256, 256, 512], [256, 256, 512, 1024]]
[515, 256, 256, 512]
[515, 256, 512, 1024]


SSD3D(
  (vfe): None
  (backbone_3d): PointNet2MSG_FPS(
    (SA_modules): ModuleList(
      (0): PointnetSAModuleMSG_FPS(
        (groupers): ModuleList(
          (0): QueryAndGroup()
          (1): QueryAndGroup()
          (2): QueryAndGroup()
        )
        (mlps): ModuleList(
          (0): Sequential(
            (0): Conv2d(4, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU()
            (3): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (4): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (5): ReLU()
            (6): Conv2d(16, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (7): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (8): ReLU()
          )
          (1): Sequential(
            (0): Conv2d(4, 16, kernel_size=(1, 1),

In [6]:
print(model)


SSD3D(
  (vfe): None
  (backbone_3d): PointNet2MSG_FPS(
    (SA_modules): ModuleList(
      (0): PointnetSAModuleMSG_FPS(
        (groupers): ModuleList(
          (0): QueryAndGroup()
          (1): QueryAndGroup()
          (2): QueryAndGroup()
        )
        (mlps): ModuleList(
          (0): Sequential(
            (0): Conv2d(4, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU()
            (3): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (4): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (5): ReLU()
            (6): Conv2d(16, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (7): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (8): ReLU()
          )
          (1): Sequential(
            (0): Conv2d(4, 16, kernel_size=(1, 1),

In [7]:
data_batch = next(iter(dataloader))

{'cls_id': [1, 1, 1, 1, 2, 2], 'cls_score': [0.99626, 0.984963, 0.662021, 0.529798, 0.63587, 0.527812], 'bbox_2d': [array([525, 172, 735, 367], dtype=int32), array([1033,  177, 1175,  217], dtype=int32), array([175, 193, 204, 206], dtype=int32), array([698, 193, 763, 243], dtype=int32), array([638, 198, 683, 245], dtype=int32), array([566, 196, 609, 238], dtype=int32)]}{'cls_id': [1, 1, 1, 1, 1, 1], 'cls_score': [0.998925, 0.995367, 0.994282, 0.966971, 0.860621, 0.529648], 'bbox_2d': [array([173, 195, 436, 352], dtype=int32), array([432, 178, 486, 212], dtype=int32), array([502, 174, 585, 249], dtype=int32), array([598, 178, 640, 212], dtype=int32), array([579, 173, 617, 205], dtype=int32), array([658, 176, 669, 185], dtype=int32)]}{'cls_id': [1, 2], 'cls_score': [0.757571, 0.722649], 'bbox_2d': [array([527, 176, 556, 198], dtype=int32), array([608, 174, 617, 192], dtype=int32)]}


{'cls_id': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2], 'cls_score': [0.996867, 0.993518, 0.992652, 0.9926

In [8]:
data_batch.keys()

dict_keys(['points', 'frame_id', 'calib', 'gt_boxes', 'road_plane', 'cls_score_feats', 'full_img_votes_1d', 'full_img_1d', 'full_img_width', 'full_img_height', 'use_lead_xyz', 'batch_size'])

In [9]:
def load_data_to_gpu(batch_dict):
    for key, val in batch_dict.items():
        if not isinstance(val, np.ndarray):
            continue
        if key in ['frame_id', 'metadata', 'calib', 'image_shape']:
            continue
        batch_dict[key] = torch.from_numpy(val).float().cuda()

load_data_to_gpu(data_batch)

In [10]:
model(data_batch)

img_ torch.Size([2, 256, 1024]) torch.Size([2, 768, 3]) torch.Size([2, 256, 3])
point_ torch.Size([2, 256, 1024]) torch.Size([2, 768, 3]) torch.Size([2, 256, 3])
fusion_ torch.Size([2, 512, 1024]) torch.Size([2, 768, 3]) torch.Size([2, 256, 3])


({'loss': tensor(221.7804, device='cuda:0', grad_fn=<AddBackward0>)},
 {'img_vote_loss': 1.2471857070922852,
  'img_center_loss_cls': 261.65966796875,
  'img_center_pos_num': 4.0,
  'img_box_res_loss': 1.097867727279663,
  'img_angle_res_loss': 0.19716744124889374,
  'img_angle_bin_loss': 2.373516321182251,
  'img_angle_loss': 2.570683717727661,
  'img_corner_loss_reg': 1.1676225662231445,
  'point_vote_loss': 0.9483085870742798,
  'point_center_loss_cls': 231.0950927734375,
  'point_center_pos_num': 5.0,
  'point_box_res_loss': 1.8554918766021729,
  'point_angle_res_loss': 0.202121764421463,
  'point_angle_bin_loss': 2.356865882873535,
  'point_angle_loss': 2.558987617492676,
  'point_corner_loss_reg': 1.4155008792877197,
  'fusion_vote_loss': 0.5982701182365417,
  'fusion_center_loss_cls': 166.75404357910156,
  'fusion_center_pos_num': 7.0,
  'fusion_box_res_loss': 1.8167188167572021,
  'fusion_angle_res_loss': 1.25178861618042,
  'fusion_angle_bin_loss': 2.43003511428833,
  'fusion_